# Thinking in tensors in PyTorch

Hands-on training  by [Piotr Migdał](https://p.migdal.pl) (2019). Version 0.4 for Uniwersytet Śląski.


## Extra: Transfer learning

Notebook by Piotr Migdał.

**Under construction**


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/stared/thinking-in-tensors-writing-in-pytorch/blob/master/extra/Image%20classification.ipynb)

No aliens vs predators, no nudity.  
Typical use case: hot dog.